In [ ]:
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
np.random.seed(1000)
# ignore information messgaes from tensorflow, but we will receieve error messages
os.environ['TFF_CPP_MIN_LOG_LEVEL'] = '2'

%matplotlib inline

In [ ]:
pickle_in = open("./resources/X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("./resources/Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

pickle_in = open("./resources/X_valid.pickle","rb")
X_valid = pickle.load(pickle_in)

pickle_in = open("./resources/Y_valid.pickle","rb")
Y_valid = pickle.load(pickle_in)

pickle_in = open("./resources/X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("./resources/Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

print(f"X_train= {X_train.shape} Y_train= {Y_train.shape}")
print(f"X_valid= {X_valid.shape} Y_valid= {Y_valid.shape}")
print(f"X_test= {X_test.shape} Y_test= {Y_test.shape}")

X_train= (1662, 227, 227, 3) Y_train= (1662,)
X_valid= (207, 227, 227, 3) Y_valid= (207,)
X_test= (210, 227, 227, 3) Y_test= (210,)


In [ ]:
mean_img = np.mean(X_train, axis=0)

**This code implements a real-time mask detection** system using a pre-trained AlexNet-inspired convolutional neural network (CNN) model. The model is loaded from a saved file, and a webcam stream captures frames in real-time. The OpenCV library is utilized to apply a Haar Cascade classifier for face detection within each frame. Detected faces are processed, and the model predicts the mask-wearing status—Mask, No Mask, or Improper Mask. The predictions are visually labeled on the video feed, outlining the face with rectangles of different colors based on the predicted class. The application runs continuously until interrupted by pressing the 'q' key, providing a live demonstration of the mask detection system in action. This real-time implementation can be applied to enforce safety measures in various settings, promoting mask compliance for public health.

In [ ]:

# Load the pre-trained model
model = tf.keras.models.load_model("./resources/alexnet_model.h5", compile=False)

# Function to preprocess the face image
def preprocess(face_img):
    face_img = cv2.resize(face_img, (227, 227))  # Resize to the size expected by your model
    face_img = face_img-mean_img
    face_img = np.expand_dims(face_img, axis=0)
    return face_img

# Start video capture from webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(frame, 1.1, 4)

    # Loop through detected faces
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        processed_face = preprocess(face_img)

        # Predict mask status
        prediction = model.predict(processed_face)
        label = np.argmax(prediction, axis=1)

        # Label the prediction
        if label == 0:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
            cv2.putText(frame, 'Improper Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
        elif label == 1:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, 'No Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        else:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Mask Detection', frame)

    # Break the loop with the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 13ms/step
